<a href="https://colab.research.google.com/github/fkonrad97/Network/blob/main/Colab/Graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [139]:
from collections import defaultdict
import pandas as pd
import numpy as np

In [140]:
# Read positions of nodes (X,Y,Z)

def positionRead(name):
    positions = pd.read_csv(name + ".csv", header=None, sep=";")
    # Remove a plus sign from the end of the number
    positions[0][0] = positions[0][0][:-1]
    positions[0] = positions[0].astype(float)    # Convert data to numerical value
    return positions

In [141]:
# Read connection table between nodes

def connectionRead(name):
    connections = pd.read_csv(name + ".csv", header=None)
    return connections

In [142]:
# List of the positions of nodes
positions = positionRead("Network/Brain_data/Brain1Positions")

# List of how nodes connected to each other
connections = connectionRead("Network/Brain_data/Brain1Connections")

C:\Users\fkonr\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [143]:
# Calculate distance between 2 points in Euclidean space
# Positions should be 3D (X,Y,Z)

def Euclidean_dist(p1, p2):
    point1 = np.array((positions[0][p1], positions[1][p1], positions[2][p1]))
    point2 = np.array((positions[0][p2], positions[1][p2], positions[2][p2]))
    return np.linalg.norm(point1 - point2)

In [144]:
# Get neighbours of 'p' from the connections dataframe 

def get_neighbours(p): 
    neighbours = []
    
    for i in range(len(positions[0])):
        if connections[i][p] == 1:
            neighbours.append(i)
    
    return neighbours

In [145]:
positions

,0,1,2
0,86.828723,55.964199,26.615948
1,68.934166,51.139614,32.673099
2,99.234568,29.370370,38.259259
3,101.797312,49.265398,28.711646
4,65.881081,61.874595,43.203784
...,...,...,...
78,125.030392,96.151471,28.293137
79,114.409524,80.682993,26.676190
80,130.959165,111.471842,12.879515
81,128.398802,94.169108,12.902796


In [146]:
connections

,0,1,2,3,4,5,6,7,8,9,...,73,74,75,76,77,78,79,80,81,82
0,0,1,1,1,1,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,1,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,1,0,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,1,0,0,0,1,1,0,0,...,0,0,0,1,0,0,1,0,0,0
4,1,1,0,0,0,1,1,1,1,0,...,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,0,0,0,0,0,0,1,0,0,1,...,1,1,1,1,1,0,1,1,1,1
79,0,0,0,1,0,0,0,1,0,0,...,0,0,1,1,1,1,0,1,1,1
80,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,0,1,1
81,0,0,0,0,0,0,0,0,0,0,...,0,1,1,1,1,1,1,1,0,1


In [147]:
def toDictionary(positions, connections):
  edges = []
  for i in range(len(connections[0])):
    for j in range(len(connections[0])):
      if i!=j and i<j and connections[i][j]==1:
        edges.append((i,j,Euclidean_dist(i,j)))
  return edges

In [148]:
class Graph():
    def __init__(self):
        """
        self.edges is a dict of all possible next nodes
        e.g. {'X': ['A', 'B', 'C', 'E'], ...}
        self.weights has all the weights between two nodes,
        with the two nodes as a tuple as the key
        e.g. {('X', 'A'): 7, ('X', 'B'): 2, ...}
        self.shortest_path is a dict of node - full distance pairs
        e.g init node is 1 -> {8: (4, 60.511292298086815)} means from init node 1 through node 4 to node 8, the full distance between node 1 and node 8 is 60.51...
        self.node2node_paths is a dict of node and node to node ditances
        e.g init node is 1 -> {8: (4, 45.166701725970064)} means from node 4 to node 8 the distance is 45.166...
        """
        self.edges = defaultdict(list)
        self.weights = {}
        self.shortest_paths = {}
        self.node2node_paths = {}
    
    def add_edge(self, from_node, to_node, weight):
        # Note: assumes edges are bi-directional
        self.edges[from_node].append(to_node)
        self.edges[to_node].append(from_node)
        self.weights[(from_node, to_node)] = weight
        self.weights[(to_node, from_node)] = weight

    def dijsktra(self, initial, end):
      # shortest paths is a dict of nodes
      # whose value is a tuple of (previous node, weight)
      current_node = initial
      visited = set()
      self.shortest_paths = {initial: (None, 0)}
      self.node2node_paths = {initial: (None, 0)}
    
      while current_node != end:
        visited.add(current_node)
        destinations = graph.edges[current_node]
        weight_to_current_node = self.shortest_paths[current_node][1]

        for next_node in destinations:
            nodeDist = graph.weights[(current_node, next_node)]
            weight = graph.weights[(current_node, next_node)] + weight_to_current_node
            if next_node not in self.shortest_paths:
                self.shortest_paths[next_node] = (current_node, weight)
                self.node2node_paths[next_node] = (current_node, nodeDist)
            else:
                current_shortest_weight = self.shortest_paths[next_node][1]
                if current_shortest_weight > weight:
                    self.shortest_paths[next_node] = (current_node, weight)
                    self.node2node_paths[next_node] = (current_node, nodeDist)
        
        next_destinations = {node: self.shortest_paths[node] for node in self.shortest_paths if node not in visited}
        if not next_destinations:
            return "Route Not Possible"
        # next node is the destination with the lowest weight
        current_node = min(next_destinations, key=lambda k: next_destinations[k][1])

    def findAll(self, initial):
      for i in range(len(connections)):
        if i!=initial:
          self.dijsktra(initial, i)
          if(len(self.shortest_paths)==len(connections[0])):
            break;

    def shortestPath(self, initial, end):
      # Work back through destinations in shortest path
      current_node = end
      path = []
      while current_node is not None:
        path.append(current_node)
        next_node = self.shortest_paths[current_node][0]
        current_node = next_node
      # Reverse path
      path = path[::-1]
      return path

In [149]:
# Initialize graph

graph = Graph()
edges = toDictionary(positions, connections)

for edge in edges:
    graph.add_edge(*edge)

In [154]:
graph.findAll(0)